This uses the materialised views created for the SQL warehouse advistor and the DBSQL cost dashboard

In [0]:
%sql
select *
  from users.steve_johansen.dbsql_cost_per_query
 where start_time >= date_sub(current_timestamp(), :history_days)
 order by query_attributed_dollars_estimation desc

In [0]:
%sql
select *
  from users.steve_johansen.warehouse_query_history
 where start_time >= date_sub(current_timestamp(), :history_days)

In [0]:
%sql
with cost as (
  select *
  from users.steve_johansen.dbsql_cost_per_query
 where start_time >= date_sub(current_timestamp(), :history_days)
), query as (
  select *
  from users.steve_johansen.warehouse_query_history
 where start_time >= date_sub(current_timestamp(), :history_days)
), workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
)
select c.query_attributed_dollars_estimation,
       coalesce(try_divide(c.query_attributed_dollars_estimation, q.ReadDataAmountInGB), 0) as cost_per_read_gb,
       coalesce(try_divide(c.query_attributed_dollars_estimation, q.WrittenDataAmountInGB), 0) as cost_per_write_gb,
       q.ReadDataAmountInGB,
       q.WrittenDataAmountInGB,
       q.FilesPrunedProportion,
       q.SpilledDataAmountInGB,
       wh.warehouse_id,
       wh.warehouse_name,
       wh.warehouse_type,
       wh.warehouse_size,
       wh.min_clusters,
       wh.max_clusters,
       wh.auto_stop_minutes,
       wh.tags,
       c.*,
       q.*
  from cost c
  join query q
    on c.statement_id = q.statement_id
  left join warehouse_scd wh
    on q.warehouse_id = wh.warehouse_id
   and q.start_time between wh.valid_from and coalesce(wh.valid_to, current_timestamp())
   and q.workspace_id = wh.workspace_id
 where c.query_attributed_dollars_estimation >= :cost_min
   and q.ReadDataAmountInGB >= :read_gb_min
   and q.WrittenDataAmountInGB >= :write_gb_min
   and q.statement_text <> 'execute immediate qry_str'
 order by c.query_attributed_dollars_estimation desc

In [0]:
%sql
with workspaces as (
     select workspace_id,
            workspace_name,
            workspace_url
       from system.access.workspaces_latest
), warehouse_scd as (
     select wh.*,
            w.workspace_name,
            w.workspace_url,
            wh.change_time as valid_from,
            lead(wh.change_time) over (
                partition by wh.workspace_id, wh.warehouse_id
                order by wh.change_time
            ) as valid_to,
            case
                when row_number() over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) = 1 then 'add'
                when lead(wh.change_time) over (partition by wh.workspace_id, wh.warehouse_id order by wh.change_time) is null then 'delete'
                else 'update'
            end as change_op
       from system.compute.warehouses wh
       left join workspaces w
         on w.workspace_id = wh.workspace_id
)
select *
  from warehouse_scd
 where warehouse_id = '862f1d757f0424f7'